# Content
* Importing Libraries
* Constants
* Preprocessing
    * Normalizing
    * Tokenizing
    * Stemming
    * Lemmatizing
* Feature Engineering
    * Bag of Words
    * FastText Word2Vec
* Model Selection

# Importing Libraries

In [189]:
from __future__ import unicode_literals

import json
import os
import numpy as np
import re
import pandas as pd
from functools import reduce
from hazm import *
from pprint import pprint

# Constants

In [190]:
data_root = 'data'

keys = ['_id', 'NewsAgency', 'newsCode', 'newsLink', 'date',
        'newsPath', 'newsPathLinks', 'title', 'rutitr', 'subtitle',
  'body', 'bodyHtml', 'tags']

valid_tags = ['اجتماعی','اقتصادی','بین الملل','حوادث','خواندنی ها و دیدنی ها',
            'داستان کوتاه','روانشناسی','سرگرمی','سلامت','سیاست خارجی','سیاسی','علمی',
            'عمومی','فرهنگی/هنری','فناوری و IT','ورزشی']

news_agencies = ['AsrIran']

# Preprocessing

### Import Dataset

In [191]:
with open(os.path.join(data_root, 'out.jsonl'), encoding='utf-8') as json_data:
    news = [json.loads(line) for line in json_data]
    news = pd.DataFrame(news)
print('Number of Datapoints: {}'.format(len(news)))

Number of Datapoints: 1000


Lets look at our data

In [192]:
news.head(2)

,NewsAgency,_id,body,bodyHtml,date,newsCode,newsLink,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,5b4f7279020eb20597f401b4,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,"<img align=""left"" class=""news_corner_image"" s...",تاریخ انتشار: ۲۱:۲۲ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621662,http://www.asriran.com/fa/news/621662,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,5b4f7279020eb20597f401b5,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,<p><br/>دادستان انتظامی مالیاتی سازمان امور م...,تاریخ انتشار: ۲۱:۱۱ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621659,http://www.asriran.com/fa/news/621659,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...


### subtitle & rutitr

In [193]:
print(news.subtitle.sample(5))
print(news.rutitr.sample())

916    اگرچه با گران شدن یکباره سوخت در کشور بدون درن...
894    قاضی هر قدر به مأموران می‌گوید بازپرس است و در...
316                                                     
130    سخنگوی یکی از گروه‌های معترض مدتی بعد اعلام کر...
307    مشتریان آدامس ایرانی در این مدت کشورهای افغانس...
Name: subtitle, dtype: object
113    
Name: rutitr, dtype: object


As we can see, there might not exist any 'subtitle' or 'rutitr', so we drop them if they do not have valuable features.

In [194]:
print("Not null 'subtitle' ",len([i for i in news.subtitle if len(i) != 0]))
print("Not null 'rutitr' ",len([i for i in news.rutitr if len(i) != 0]))

Not null 'subtitle'  569
Not null 'rutitr'  50


So based on information we got here, we know that these columns can help us, so we consider them.

### Drop Useless Columns

But it is clear for us, `date`,`newsCode`, `newsLink`,`bodyHtml` and `_id` are useless features. So we remove them from our dataset.


In [195]:
news = news.drop(['_id','date','newsCode','newsLink','bodyHtml'], axis=1)

In [196]:
news.head(5)

,NewsAgency,body,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,صفحه نخست » اقتصادی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اقت...",,,"{'اوپک': '/fa/tag/1/اوپک', 'نفت': '/fa/tag/1/ن...",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...
3,AsrIran,رئیس فراکسیون فرهنگیان مجلس خطاب به وزیر آموزش...,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'آموزش و پرورش': '/fa/tag/1/آموزش و پرورش', '...",حاجی‌بابایی به بطحایی:آقای وزیر در اطلاع‌رسانی...
4,AsrIran,رئیس صندوق بین&zwnj;المللی پول در آستانه نشست ...,صفحه نخست » بین الملل,"{'صفحه نخست': '/fa/archive?service_id=1', 'بین...",,,{'صندوق بین المللی پول': '/fa/tag/1/صندوق بین ...,رئیس صندوق بین‌المللی پول: آمریکا از جنگ تعرفه...


### newsPath & newsPathLinks

In [197]:
newspathlinks_tags = list(set([list(x.keys())[1] for x in news.newsPathLinks]))
newspath_tags = list(set(x.split(' » ')[1] for x in news.newsPath))
print("news path links and news path show the same thing? => ", newspath_tags == newspathlinks_tags)
valid_tags = newspath_tags
pd.DataFrame([valid_tags])

news path links and news path show the same thing? =>  True


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,بین الملل,علمی,خواندنی ها و دیدنی ها,عمومی,ورزشی,سلامت,اقتصادی,حوادث,سیاسی,داستان کوتاه,سیاست خارجی,سرگرمی,فرهنگی/هنری,دانلود,روانشناسی,فناوری و IT,اجتماعی


Note: we can remove `newsPathLinks` because it is exactly duplicate of `newsPath` and update `newsPath` column in dataframe with just one keyword

In [198]:
news = news.drop(['newsPathLinks'], axis=1)
news.head(2)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...


In [ ]:
news.loc[:,'newsPath'] = list((x.split(' » ')[1] for x in news.newsPath))

### NewsAgency

In [199]:
news_agencies = list(news.NewsAgency.unique())
pd.DataFrame([news_agencies])

,0
0,AsrIran


### tags

In [200]:
def tag_extractor(tags_dict):
    """
    gets a tags dictionary and finds unique tags in collection of values and keys
    
    ::params tags_dict : 
    """
    keys = list(set(tags_dict.keys()))
    values = { v.split('/')[-1] for v in set(tags_dict.values())}
    [values.add(i) for i in keys]
    return list(values)

print('not processed tags_dict',news.tags[0])
print('processesd tags_dict',tag_extractor(news.tags[0]))

not processed tags_dict {'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/fa/tag/1/افتخاری'}
processesd tags_dict ['استقلال', 'افتخاری']


Now we replace `tags` column with extract values from `tag_extractor` function.

In [201]:
news.loc[:, 'tags'] = [tag_extractor(tag) for tag in news.tags]

In [202]:
news.head(3)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,,,"[استقلال, افتخاری]",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,,,"[مالیات, دادستان]",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,صفحه نخست » اقتصادی,,,"[اوپک, نفت]",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...


## Normalizing

In [38]:
normalizer = Normalizer()
news['body'] = news['body'].apply(normalizer.normalize)
news['rutitr'] = news['rutitr'].apply(normalizer.normalize)
news['subtitle'] = news['subtitle'].apply(normalizer.normalize)
news['title'] = news['title'].apply(normalizer.normalize)

## Tokenizing

In [40]:
def tokenize(phrase):
    sentences = sent_tokenize(phrase)
    if len(sentences) > 1:
        words = reduce(np.append, [word_tokenize(sentence) for sentence in sentences])
    elif len(sentences) == 1:
        words = word_tokenize(sentences[0])
    else:
        words = None
    return words

In [41]:
news['body'] = news['body'].apply(tokenize)
news['rutitr'] = news['rutitr'].apply(tokenize)
news['subtitle'] = news['subtitle'].apply(tokenize)
news['title'] = news['title'].apply(tokenize)

## Stemming

In [42]:
stemmer = Stemmer()
stem = lambda s: [stemmer.stem(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(stem)
news['rutitr'] = news['rutitr'].apply(stem)
news['subtitle'] = news['subtitle'].apply(stem)
news['title'] = news['title'].apply(stem)

## Lemmatizing

In [43]:
lemmatizer = Lemmatizer()
lemmatize = lambda s: [lemmatizer.lemmatize(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(lemmatize)
news['rutitr'] = news['rutitr'].apply(lemmatize)
news['subtitle'] = news['subtitle'].apply(lemmatize)
news['title'] = news['title'].apply(lemmatize)

# Feature Engineering

# Model Selection

## Naive Bayes

## Logistic Regression

## SVM

## Simple ANN

## CNN

## RNN

## CNN + RNN